In [1]:
import trimesh as tm
import numpy as np
import pyvista as pv
import os
import topogenesis as tg
import pickle as pk

In [2]:
context_path = os.path.relpath('../data/movedcontext.obj')
context_mesh = tm.load(context_path)

env_lat_path = os.path.relpath('../data/macrovoxels.csv')
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [3]:
# create a sphere 
sphere = tm.creation.icosphere(subdivisions= 3, radius= 300.0)

# extract vertices from sphere
sphere_vertices = sphere.vertices

# keep only positive vertices
skydome = sphere_vertices[sphere_vertices[:,2] > 0.0]

# convert to array
skydome = np.array(skydome)

# shift skydome down to ground height --> shift down with minimum height of skydome points +10 and height offset of lattice and environment
skydome[:,2] += -min(skydome[:,2]-10) + min(envelope_lattice.centroids[:,2])

In [4]:
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# add the sky vectors
p.add_points(skydome, color='#0013ff')

# add context
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(999.772514064959, 996.4703958584882, 1105.5574518419928),
 (3.302118206470709, 0.0, 109.08705598350447),
 (0.0, 0.0, 1.0)]

In [5]:
# save skydome to pk
pk.dump(skydome, open("../data/skydome.pk", "wb"))